<a href="https://colab.research.google.com/github/DavideRutigliano/frigate-models/blob/main/notebooks/YOLO_NAS_Pretrained_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [1]:
%%capture
!pip install super-gradients
!git clone https://github.com/Deci-AI/super-gradients.git /content/super_gradients_folder
!sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/pretrained_models.py
!sed -i 's/sghub.deci.ai/sg-hub-nv.s3.amazonaws.com/' /content/super_gradients_folder/src/super_gradients/training/utils/checkpoint_utils.py
!pip install -e /content/super_gradients_folder
!pip install onnxruntime
!pip install pytorch-quantization==2.1.2 --extra-index-url https://pypi.ngc.nvidia.com
!pip install datasets

## Select model

In [8]:
model_name = 'yolonas_s' #@param ["yolonas_s", "yolonas_m", "yolonas_l"]
quantization = 'uint8' #@param ["fp16", "uint8"]
input_width = 320 #@param {type:"slider", min:320, max:640, step:320}
input_height = 320 #@param {type:"slider", min:320, max:640, step:320}

MODEL_FILENAME = f"{model_name}_{quantization}_{input_width}x{input_height}.onnx"

In [9]:
from super_gradients.common.object_names import Models
from super_gradients.training import models

MODEL_NAMES = {
    "yolonas_s": Models.YOLO_NAS_S,
    "yolonas_m": Models.YOLO_NAS_M,
    "yolonas_l": Models.YOLO_NAS_L,
}

model = models.get(MODEL_NAMES[model_name], pretrained_weights="coco")
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, input_height, input_width])

[2025-03-14 16:19:17] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2025-03-14 16:19:17] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2025-03-14 16:19:17] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


In [20]:
from super_gradients.conversion import DetectionOutputFormatMode
from super_gradients.conversion.conversion_enums import ExportQuantizationMode

# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from super_gradients.training.utils.collate_fn import DetectionCollateFN

# NUM_WORKERS = 0
# BATCH_SIZE = 1

# dataloader_params = {
#     "shuffle": False,
#     "batch_size": BATCH_SIZE,
#     "drop_last": False,
#     "pin_memory": True,
#     "collate_fn": DetectionCollateFN(),
#     "num_workers": NUM_WORKERS,
#     "persistent_workers": NUM_WORKERS > 0,
# }

# calibration_dataset = load_dataset("detection-datasets/coco", split="val")
# calibration_loader = DataLoader(calibration_dataset, **dataloader_params)

model.export(
  MODEL_FILENAME,
  input_image_shape=(input_height, input_width),
  num_pre_nms_predictions=300,
  max_predictions_per_image=5,
  nms_threshold=0.7,
  confidence_threshold=0.4,
  # calibration_loader=calibration_loader,
  quantization_mode=(
      ExportQuantizationMode.FP16 if quantization == "fp16" else ExportQuantizationMode.INT8
  ),
  output_predictions_format=DetectionOutputFormatMode.FLAT_FORMAT,
)

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

DatasetItemsException: `DetectionCollateFN` only supports Datasets that return a tuple ('image', 'targets'), but got a tuple of len=1

In [ ]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.uint8)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"input": dummy_input})

In [ ]:
from google.colab import files

files.download(MODEL_FILENAME)